In [5]:
# packages

import numpy as np
import cobra
from cobra import Reaction, Metabolite, Model

In [2]:
from os.path import dirname, join, pardir

from cobra.io import read_sbml_model

from dfba import DfbaModel, ExchangeFlux, KineticVariable

ModuleNotFoundError: No module named 'dfba'

In [3]:
model = read_sbml_model("iJN678mod.xml")
model

Name,iJN678
Memory address,0x0137feed8fd0
Number of metabolites,796
Number of reactions,865
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_SynHetero - 1.0*BIOMASS_Ec_SynHetero_reverse_5d8af
Compartments,"cytosol, periplasm, thylakoid, extracellular space"


Enhancing Squalene Production in Saccharomyces cerevisiae by Metabolic Engineering and Random Mutagenesis

Practical industrial squalene production, however, still needs further enhancement of production by S. cerevisiae. Insufficient flux through the MVA pathway is an important limiting factor and the natural biosynthesis rate of the essential precursor, acetyl-CoA is insufficient for high squalene production.

The composition of YPD medium was as follows: 20 g/L glucose, 10 g/L yeast extract, and 20 g/L peptone.

Production of squalene by squalene synthases and their truncated mutants in Escherichia coli

Introduction of a green algal squalene synthase enhances squalene accumulation in a strain of Synechocystis sp. PCC 6803

Synechocystis cultures were grown at 30°C under a light intensity of 20 μmol photons m-2 s-1

Growth kinetics and mathematical modeling of Synechocystis sp. PCC 6803 under flashing light


Multi-component kinetics for the growth of the cyanobacterium Synechocystis sp. PCC6803

maximum specific growth rate (μmax) = 2.8/d, half-maximum-rate light irradiance (KL) = 11 W/m2, half-inhibition-rate light irradiance (KL,I) = 39 W/m2, and half-maximum-rate concentration for inorganic carbon (KS,Ci) = 0.5 mgC/L, half-maximum-rate concentration for inorganic nitrogen (KS,Ni) = 1.4 mgN/L, and half-maximum-rate concentration for inorganic phosphorus (KS,Pi) = 0.06 mgP/L.

To gain the advantages of its high maximum specific growth rate, PCC6803 needs to have moderate light illumination (7–62 W/m2 for μsyn ≥ 1/d) and relatively high nutrient concentrations: Ni ≥ 2.3 mgN/L, Pi ≥ 0.1 mgP/L, and Ci ≥ 1.0 mgC/L.

μsyn is the specific growth rate, μmax is the maximum specific growth rate, KS is half-maximum-rate concentration for a given nutrient, KL is the half-maximum-rate light irradiance, KL,I is the half-inhibition-rate light irradiance, LI is the light illumination to which the microorganisms are exposed, KS,Ci, is the half-maximum-rate concentrations for inorganic carbon, KS,Ni is the half-maximum-rate concentrations for inorganic nitrogen, KS,Pi is the half-maximum-rate concentrations for inorganic phosphorus, Ci is the total concentration of the carbon, Ni is the total concentration of the nitrogen, Pi is the total concentration of the phosphorus, and square bracket with subscript ‘min’ indicates the minimum value among the individual mathematical expressions separated by commas.

Non-limiting Ni, Pi, and other nutrients were supplied using standard BG-11 medium to which we added supplemental Pi using a semi-batch mode of operation. .5 g NaNO3, 40 mg K2HPO4·3H2O, 75 mg MgSO4·7H2O, 36 mg CaCl2·2H2O, 6 mg citric acid, 6 mg ferric ammonium citrate, 1 mg EDTA disodium salt, and 1 ml mixed trace metal solution.

CO2 content (ppmva) = 387. C supply rate = 1,04 mg/LR/min

Monod kinetics for N, P and C.

In [5]:
print(model.objective)

Maximize
1.0*BIOMASS_Ec_SynHetero - 1.0*BIOMASS_Ec_SynHetero_reverse_5d8af


In [ ]:
path_to_model = join(pardir, "sbml-models", "iJN678mod.xml")
fba_model = read_sbml_model(path_to_model)
fba_model.solver = "glpk"
dfba_model = DfbaModel(fba_model)

In [ ]:
# kynetic variables

X = KineticVariable("Biomass")
Pht = KineticVariable("Photons")
CO2 = KineticVariable("CO2", initial_condition=0.5)
Sql = KineticVariable("Squalene")
N = KineticVariable("Nitrogen")
F = KineticVariable("Phosphor)

dfba_model.add_kinetic_variables([X, Pht, CO2, Sql, N, P])

In [ ]:
# exchange fluxes

mu = ExchangeFlux("BiomassEcoli")
v_P = ExchangeFlux("EX_photon_e")
v_C = ExchangeFlux("EX_co2_e")
v_S = ExchangeFlux("EX_sql_e")
v_N = ExchangeFlux("EX_nh4_e")
v_F = ExchangeFlux("EX_pi_e")

dfba_model.add_exchange_fluxes([mu, v_P, v_C, v_S, v_N, v_F])

A kinetic expression, in the form of a differential equation, like the one shown below, has to be added to each dynamic variable, in order to account for variations in the concentration of each compound, where Cj represents a given compound, and Vj is the specific rate of consumption by biomass:

In [ ]:
# Rhs expression

dfba_model.add_rhs_expression("Biomass", mu * X)
dfba_model.add_rhs_expression("Photons", v_G * 180.1559 * X / 1000.0)
dfba_model.add_rhs_expression("CO2", v_Z * 150.13 * X / 1000.0)
dfba_model.add_rhs_expression("Squalene", v_O * 16.0 * X / 1000.0)
dfba_model.add_rhs_expression("Ammonia", v_N * 17.031/1000 * X)
dfba_model.add_rhs_expression("Phosphate", v_F * 94.9714/1000 * X)

In [ ]:
# Lower/Upper bound expressions

vmax_C = 1.5 # [h^-1]
Ks_C = 10.6 # mg/L
dfba_model.add_exchange_flux_lb("EX_co2_e", vmax_C * (CO2 / (Ks_C), CO2)

vmax_N = 1.6 # [h^-1]
Ks_N = 1.4 # mg/L
dfba_model.add_exchange_flux_lb("EX_nh4_e", vmax_N * (N / (Ks_N), N)

vmax_F = 1.6 # [h^-1]
Ks_F = 0.060 # mmg/L
dfba_model.add_exchange_flux_lb("EX_pi_e", vmax_F * (Phs / (Ks_N + Phs + (Phs * Phs / Ki_N))), Phs)

In [ ]:
# Initial conditions for the model

dfba_model.add_initial_conditions(
    {
        "Biomass": 0.3,
        "CO2": 10.5,
        "Nitrogen": 0.5,
        "Phosphate": 0.3,
        "Squalene": 0.0,
        "Photoms": 20.0,
    }
)
concentrations, trajectories = dfba_model.simulate(0.0, 25.0, 0.1, ["EX_co2_e", "EX_nh4_e", "EX_pi_e"])

In [ ]:
# plot the results

from dfba.plot.plotly import *
import plotly.io as pio

fig = plot_concentrations(concentrations)
fig.show()

fig = plot_trajectories(trajectories)
fig.show()